In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
from datasets import load_dataset

dataset = load_dataset('UCSD-GENIE/ClimaQA', data_files="climaqa_gold/ffq/ffq_benchmark.csv")
dataset = dataset['train']


ffq_df = dataset.to_pandas()
ffq_df = ffq_df[ffq_df['Validation'] == True]

In [4]:
import random
from qa import qa_utils
from qa.qa_agent import QAAgent
from qa.qa_fewshot_agent import QAFewshotAgent
from qa.qa_rag_agent import QARagAgent
from llm.openai_llm import OpenAIILLM
from llm.together_ai_llm import TogetherAILLM

ind = random.randint(0, len(ffq_df)-1)
question = list(ffq_df['Question'])[ind]
reference_answer = list(ffq_df['Answer'])[ind]

# Default LLM Agents
qa_agents = [
    QAAgent(OpenAIILLM('gpt-3.5-turbo')),
    QAAgent(OpenAIILLM('gpt-4o')),
    QAAgent(TogetherAILLM('llama3-70b')),
    QAAgent(TogetherAILLM('mixtral-8x22b')),
    QAAgent(TogetherAILLM('gemma-27b'))
]

# LLM Agents that answer with Few-shot prompting
# qa_agents = [
#     QAFewshotAgent(OpenAIILLM('gpt-3.5-turbo')),
#     QAFewshotAgent(OpenAIILLM('gpt-4o')),
#     QAFewshotAgent(TogetherAILLM('llama3-70b')),
#     QAFewshotAgent(TogetherAILLM('mixtral-7b')),
#     QAFewshotAgent(TogetherAILLM('gemma-27b'))
# ]

# LLM Agents that answer with RAG prompting. Need to create a chroma vectordb for these to work
# qa_agents = [
#     QARagAgent(OpenAIILLM('gpt-3.5-turbo')),
#     QARagAgent(OpenAIILLM('gpt-4o')),
#     QARagAgent(TogetherAILLM('llama3-70b')),
#     QARagAgent(TogetherAILLM('mixtral-8x22b')),
#     QARagAgent(TogetherAILLM('gemma-27b'))
# ]

qa_utils.get_metrics_for_question(qa_agents, question, reference_answer)

Quesition: How would the climate be impacted if there was a significant decrease in anthropogenic aerosol particles affecting cloud droplet concentrations?
Reference Answer: If there was a significant decrease in anthropogenic aerosol particles affecting cloud droplet concentrations, it would lead to a decrease in cloud albedo and a reduction in the fundamental cooling effect of aerosol via the interaction with liquid clouds.

LLM: gpt-3.5-turbo
	Generated Answer: A significant decrease in anthropogenic aerosol particles affecting cloud droplet concentrations would likely lead to reduced cloud cover and potentially contribute to a warming effect on the climate due to less sunlight being reflected back into space.

	Bleu: 0.656
	Bert: 0.926
	FA Score: 0.898
LLM: gpt-4o
	Generated Answer: A significant decrease in anthropogenic aerosol particles would likely lead to less reflective clouds, which could reduce the cooling effect they have on the Earth's climate, potentially contributing to

In [ ]:
import os

result_dir = 'results/freeform'

num_trials = 3
for i in range(num_trials):
    print(f'\nRunning trial {i+1}\n')
    trial_dir = os.path.join(result_dir, f'trial_{i+1}')
    for agent in qa_agents:
        print(f'Answering with {agent.name}')
        qa_utils.generate_and_save_answer(agent, ffq_df, trial_dir)

In [ ]:
from qa import qa_utils

qa_utils.evaluate_result(result_dir)